In [2]:
%reload_ext autoreload
%autoreload 2

import sys
from pathlib import Path 
current_path = Path().resolve()
abs_path = str(current_path.parent)
sys.path.append(abs_path)

import warnings
warnings.filterwarnings('ignore')

from collections import defaultdict

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import plotly.graph_objs as go
import plotly.plotly as py
import plotly.offline as offline
offline.init_notebook_mode(connected=True)

import sqlalchemy as sa
from datetime import datetime as dt
import pytz
import osmnx as ox
import ast
import operator

RAW_PATH = current_path.parent / 'data' / 'raw'
OUTPUT_PATH = current_path.parent / 'data' / 'output'

In [11]:
import scripts.waze_flood as wf

# Downloading Data

## Downloanding Cities Metadata

In [15]:
#cities = wa.get_cities_metadata()

## Donwloading São Paulo Enchentes

### Tentativa por conexão direta com Athena

In [1]:
from pyathena import connect

conn = connect(s3_staging_dir='s3://aws-athena-queries-result-east-2/join-pems-waze',
               region_name='us-east-2')

In [4]:
query = """
        SELECT *,
                 pub_utc_date AT TIME ZONE 'America/Sao_Paulo' AS local_time
        FROM cities.br_saopaulo_waze_alerts
        WHERE day = 4
                AND month = 2
                AND subtype = 'HAZARD_WEATHER_FLOOD';"""

In [5]:
df = pd.read_sql(query, conn)

### Upload do csv para servidor e leitura dos dados raw

In [162]:
df = pd.read_csv(RAW_PATH/"sp-19-02-04-enchentes.csv")

In [163]:
df.head()

uuid  \
0  1502b430-6a10-30a5-838f-8e2558f1278d   
1  c0d05331-f674-38e1-be03-e3e381cc56f6   
2  12f179a6-2785-3464-b7ba-e3bdc97f011c   
3  aca8e63c-0571-3ad1-a49f-67b3a96bf1ec   
4  18881355-0b03-3f59-8471-d10f20a2a767   

                                  start_time  \
0  2019-02-04 04:32:00.000 America/Sao_Paulo   
1  2019-02-04 06:14:00.000 America/Sao_Paulo   
2  2019-02-04 05:14:00.000 America/Sao_Paulo   
3  2019-02-04 06:14:00.000 America/Sao_Paulo   
4  2019-02-04 02:56:00.000 America/Sao_Paulo   

                                    end_time  location_x  location_y  \
0  2019-02-04 05:32:00.000 America/Sao_Paulo  -46.597846  -23.589049   
1  2019-02-04 07:23:00.000 America/Sao_Paulo  -46.627209  -23.542519   
2  2019-02-04 07:38:00.000 America/Sao_Paulo  -46.600856  -23.620926   
3  2019-02-04 07:36:00.000 America/Sao_Paulo  -46.496549  -23.582749   
4  2019-02-04 03:56:00.000 America/Sao_Paulo  -46.620492  -23.552413   

   reliability_min  reliability_max  confidence_min  confidence_max  thumbs_up  
0                5                5               0               0          0  
1                5                6               0               0          1  
2                5               10               0               5         10  
3                5                6               0               0          0  
4                5                5               0               0          0

### Time Range

In [164]:
wf.get_timerange(df)

0    2019-02-03 21:57:00.000 America/Sao_Paulo
1    2019-02-04 21:56:00.000 America/Sao_Paulo
dtype: object

## OSM Graph

In [18]:
#cities = wa.download_osm_graph(city, cities)

[DEBUG] /home/master/cts/cities/waze-tools/wazetools/data/data_transform.py:68 get_osm(): Fetching data from cache
[DEBUG] /home/master/cts/cities/waze-tools/wazetools/data/data_transform.py:68 get_osm(): Fetching data from cache in 2.69 s


# Data pre-processing

## Clustering Enchentes

In [165]:
df['start_time'] = df['start_time'].apply(lambda x: dt.strptime(x, '%Y-%m-%d %H:%M:%S.000 America/Sao_Paulo'))
df['end_time'] = df['end_time'].apply(lambda x: dt.strptime(x, '%Y-%m-%d %H:%M:%S.000 America/Sao_Paulo'))

In [166]:
clusters_dict = wa.clustering(df, eps_spatial=200, eps_temporal=3600)

In [167]:
clusters_bkup = clusters_dict.copy()

In [168]:
clusters = pd.DataFrame(clusters_dict).T.drop(columns=['uuid list']).rename_axis('cluster')

In [169]:
clusters['interactions'] = clusters['thumbs_up'] + clusters['uuid count']

In [170]:
clusters.head(1)

confidence_max confidence_min             end_time location_x  \
cluster                                                                 
1                    0              0  2019-02-04 05:32:00   -46.5978   

        location_y reliability_max reliability_min           start_time  \
cluster                                                                   
1          -23.589               5               5  2019-02-04 04:32:00   

        thumbs_up uuid count interactions  
cluster                                    
1               0          1            1

## Accidents to shapely

In [171]:
wf.treat_alert_points(clusters)

In [172]:
clusters.head(1)

confidence_max confidence_min             end_time location_x  \
cluster                                                                 
1                    0              0  2019-02-04 05:32:00   -46.5978   

        location_y reliability_max reliability_min           start_time  \
cluster                                                                   
1          -23.589               5               5  2019-02-04 04:32:00   

        thumbs_up uuid count interactions  \
cluster                                     
1               0          1            1   

                                         point  
cluster                                         
1        POINT (-46.59784599999999 -23.589049)

## Match roads to points

# Ranking Clusters by Interactions

In [173]:
total_interactions = clusters['interactions'].sum()
total_alerts = clusters['uuid count'].sum()

In [174]:
total_interactions

1720

In [175]:
total_alerts

559

In [176]:
clusters['pareto'] = clusters['interactions'].sort_values(ascending=False).cumsum() * 100 / total_interactions

In [193]:
clusters_pareto_sorted = pd.DataFrame(clusters.sort_values(by='pareto'))
clusters_pareto_sorted.head()

confidence_max confidence_min             end_time location_x  \
cluster                                                                 
21                   5              0  2019-02-04 11:53:00     -46.63   
8                    4              0  2019-02-04 15:33:00   -46.5592   
67                   5              0  2019-02-04 07:35:00   -46.6933   
186                  5              0  2019-02-04 10:20:00    -46.693   
192                  5              0  2019-02-04 09:14:00   -46.6758   

        location_y reliability_max reliability_min           start_time  \
cluster                                                                   
21        -23.5195              10               5  2019-02-04 02:36:00   
8         -23.5104              10               5  2019-02-04 06:25:00   
67        -23.5978              10               5  2019-02-04 04:55:00   
186       -23.6143              10               5  2019-02-04 04:54:00   
192       -23.6271              10               5  2019-02-04 05:29:00   

        thumbs_up uuid count interactions  \
cluster                                     
21             83         31          114   
8              31         41           72   
67             54          3           57   
186            45          6           51   
192            39          2           41   

                                                 point   pareto  
cluster                                                          
21         POINT (-46.629964964799 -23.51946167009796)  6.62791  
8        POINT (-46.55923441955649 -23.51041344798837)   10.814  
67       POINT (-46.69331008391696 -23.59779010679273)  14.1279  
186      POINT (-46.69295849999998 -23.61429099999999)   17.093  
192      POINT (-46.67576243220336 -23.62707160169494)  19.4767

In [194]:
clusters_pareto_sorted.to_csv(OUTPUT_PATH/'clusters_pareto.csv')

# Bounding-Box Macro Regiões

In [203]:
def generate_bbox(clusters, cluster_lower_y, cluster_left_x, cluster_upper_y, cluster_right_x):
    bbox = {}
    bbox['lower_y'] = clusters.loc[cluster_lower_y, 'location_y']
    bbox['left_x'] = clusters.loc[cluster_left_x, 'location_x']
    bbox['upper_y'] = clusters.loc[cluster_upper_y, 'location_y']
    bbox['rigth_x'] = clusters.loc[cluster_right_x, 'location_x']
    return bbox

In [204]:
def clusters_in_bbox(clusters, bbox):
    lower_y = bbox['lower_y']
    left_x = bbox['left_x']
    upper_y = bbox['upper_y']
    rigth_x = bbox['rigth_x']
    return clusters[(left_x <= clusters['location_x']) & (clusters['location_x'] <= rigth_x) & (lower_y <= clusters['location_y']) & (clusters['location_y'] <= upper_y)]

## Região 1 (zona Norte/Leste - Tiete)

In [179]:
# lower_y = cluster 44
# left_x = cluster 64
# upper_y = cluster 45
# rigth_x = cluster 166

In [205]:
bbox1 = generate_bbox(clusters, 44,64,45,166)

In [208]:
clusters_reg1 = clusters_in_bbox(clusters, bbox1)

In [210]:
total_interactions_reg1 = clusters_reg1['interactions'].sum()
print(total_interactions_reg1)
print(total_interactions_reg1 / total_interactions *100)

342
19.883720930232556


In [211]:
clusters_reg1.to_csv(OUTPUT_PATH/'clusters_reg1.csv')

## Região 2 (zona Oeste - Pinheiros)

In [ ]:
# lower_y = cluster 186
# left_x = cluster 178
# upper_y = cluster 171
# rigth_x = cluster 186

In [222]:
bbox2 = generate_bbox(clusters, 186, 178, 171, 186)

In [223]:
clusters_reg2 = clusters_in_bbox(clusters, bbox2)

In [224]:
total_interactions_reg2 = clusters_reg2['interactions'].sum()

In [225]:
print(total_interactions_reg2)
print(total_interactions_reg2 / total_interactions *100)

149
8.662790697674419


In [226]:
clusters_reg2.to_csv(OUTPUT_PATH/'clusters_reg2.csv')

## Região 3 (Av. Jornalista Roberto Marinho)

In [184]:
# lower_y = cluster 141
# left_x = cluster 47
# upper_y = cluster 47
# rigth_x = cluster 141

In [212]:
bbox3 = generate_bbox(clusters, 141, 47, 47, 141)

In [213]:
clusters_reg3 = clusters_in_bbox(clusters, bbox3)

In [214]:
total_interactions_reg3 = clusters_reg3['interactions'].sum()

In [215]:
print(total_interactions_reg3)
print(total_interactions_reg3 / total_interactions *100)

51
2.9651162790697674


## Região 4 (Rua Miguel Yunes)

In [184]:
# lower_y = cluster 77
# left_x = cluster 39
# upper_y = cluster 39
# rigth_x = cluster 77

In [227]:
bbox4 = generate_bbox(clusters, 77, 39, 39, 77)

In [229]:
clusters_reg4 = clusters_in_bbox(clusters, bbox4)

In [230]:
total_interactions_reg4 = clusters_reg4['interactions'].sum()

In [231]:
print(total_interactions_reg4)
print(total_interactions_reg4 / total_interactions *100)

87
5.058139534883721


In [232]:
clusters_reg4.to_csv(OUTPUT_PATH/'clusters_reg4.csv')

## Região 5 (Centro)

In [184]:
# lower_y = cluster 150
# left_x = cluster 66
# upper_y = cluster 66
# rigth_x = cluster 159

In [256]:
bbox5 = generate_bbox(clusters, 150, 66, 66, 159)

In [257]:
clusters_reg5 = clusters_in_bbox(clusters, bbox5)

In [258]:
total_interactions_reg5 = clusters_reg5['interactions'].sum()

In [259]:
print(total_interactions_reg5)
print(total_interactions_reg5 / total_interactions *100)

61
3.5465116279069764


In [260]:
clusters_reg5.to_csv(OUTPUT_PATH/'clusters_reg5.csv')

## Região 6 (Rodovia Anchieta)

In [184]:
# lower_y = cluster 6
# left_x = cluster 217
# upper_y = cluster 217
# rigth_x = cluster 3

In [237]:
bbox6 = generate_bbox(clusters, 6, 217, 217, 3)

In [238]:
clusters_reg6 = clusters_in_bbox(clusters, bbox6)

In [239]:
total_interactions_reg6 = clusters_reg6['interactions'].sum()

In [240]:
print(total_interactions_reg6)
print(total_interactions_reg6 / total_interactions *100)

34
1.9767441860465116


In [241]:
clusters_reg6.to_csv(OUTPUT_PATH/'clusters_reg6.csv')

## Região 7 (Av. Paulo Freire + Av. Condessa Elizabeth de Robiano)

In [184]:
# lower_y = cluster 36
# left_x = cluster 117
# upper_y = cluster 8
# rigth_x = cluster 36

In [242]:
bbox7 = generate_bbox(clusters, 36, 117, 8, 36)

In [243]:
clusters_reg7 = clusters_in_bbox(clusters, bbox7)

In [244]:
total_interactions_reg7 = clusters_reg7['interactions'].sum()

In [245]:
print(total_interactions_reg7)
print(total_interactions_reg7 / total_interactions *100)

165
9.593023255813954


In [246]:
clusters_reg7.to_csv(OUTPUT_PATH/'clusters_reg7.csv')

## Região 8 (Av. Doutor Luís Ayres)

In [184]:
# lower_y = cluster 74
# left_x = cluster 132
# upper_y = cluster 132
# rigth_x = cluster 74

In [251]:
bbox8 = generate_bbox(clusters, 74, 132, 132, 74)

In [252]:
clusters_reg8 = clusters_in_bbox(clusters, bbox8)

In [253]:
total_interactions_reg8 = clusters_reg8['interactions'].sum()

In [254]:
print(total_interactions_reg8)
print(total_interactions_reg8 / total_interactions *100)

40
2.3255813953488373


In [255]:
clusters_reg8.to_csv(OUTPUT_PATH/'clusters_reg8.csv')

## Região 3 (Av. Jornalista Roberto Marinho)

In [184]:
# lower_y = cluster 141
# left_x = cluster 47
# upper_y = cluster 47
# rigth_x = cluster 141

In [212]:
bbox9 = generate_bbox(clusters, 141, 47, 47, 141)

In [213]:
clusters_reg9 = clusters_in_bbox(clusters, bbox9)

In [214]:
total_interactions_reg9 = clusters_reg9['interactions'].sum()

In [215]:
print(total_interactions_reg9)
print(total_interactions_reg9 / total_interactions *100)

51
2.9651162790697674


In [216]:
clusters_reg9.to_csv(OUTPUT_PATH/'clusters_reg9.csv')

# Irreg

In [153]:
def point_in_bbox(x,y, bbox):
    lower_y = bbox['lower_y']
    left_x = bbox['left_x']
    upper_y = bbox['upper_y']
    rigth_x = bbox['rigth_x']
    return (left_x <= x) & (x <= rigth_x) & (lower_y <= y) & (y <= upper_y)

In [155]:
def line_intersect_bbox(line_str, bbox):
    line_dict = ast.literal_eval(line_str)
    coords = line_dict['coordinates']
    for point in coords:
        x = point[0]
        y = point[1]
        if point_in_bbox(x,y, bbox):
            return True
    return False

In [261]:
irreg_data = pd.read_csv(RAW_PATH/"irregularities.csv")

In [262]:
irreg_data['bbox1'] = irreg_data['irre_geojson'].apply(lambda x: line_intersect_bbox(x, bbox1))

In [263]:
irreg_data['bbox2'] = irreg_data['irre_geojson'].apply(lambda x: line_intersect_bbox(x, bbox2))

In [264]:
irreg_data['bbox3'] = irreg_data['irre_geojson'].apply(lambda x: line_intersect_bbox(x, bbox3))

In [265]:
irreg_data['bbox4'] = irreg_data['irre_geojson'].apply(lambda x: line_intersect_bbox(x, bbox4))

In [266]:
irreg_data['bbox5'] = irreg_data['irre_geojson'].apply(lambda x: line_intersect_bbox(x, bbox5))

In [267]:
irreg_data['bbox6'] = irreg_data['irre_geojson'].apply(lambda x: line_intersect_bbox(x, bbox6))

In [268]:
irreg_data['bbox7'] = irreg_data['irre_geojson'].apply(lambda x: line_intersect_bbox(x, bbox7))

In [269]:
irreg_data['bbox8'] = irreg_data['irre_geojson'].apply(lambda x: line_intersect_bbox(x, bbox8))

In [277]:
irreg_data.head()

id  min_hour  max_hour                               street  length  \
0  162402419        11        11                      Marg. Pinheiros    5899   
1  162401047         8         8       SP-348 S Rod. dos Bandeirantes    3846   
2  162406068        17        17  SP-280 O Rod. Pres. Castello Branco    6678   
3  162406236        18        18           Pista Expressa Marg. Tietê    7855   
4  162400697         8         8        Av. Morvan Dias de Figueiredo    4354   

   nthumbsup  driverscount                                       irre_geojson  \
0         59          3050  {"type":"LineString", "coordinates":[[-46.7272...   
1         57          1007  {"type":"LineString", "coordinates":[[-46.7406...   
2         45           343  {"type":"LineString", "coordinates":[[-46.7995...   
3         39          1825  {"type":"LineString", "coordinates":[[-46.6325...   
4         34          2167  {"type":"LineString", "coordinates":[[-46.5910...   

   bbox1  bbox2  bbox3  bbox4  bbox5  bbox6  bbox7  bbox8  
0  False  False  False  False  False  False  False  False  
1  False  False  False  False  False  False  False  False  
2  False  False  False  False  False  False  False  False  
3   True  False  False  False  False  False   True  False  
4   True  False  False  False  False  False  False  False

In [319]:
irreg_data.to_csv(OUTPUT_PATH/'irreg_jams_by_region.csv')

In [315]:
jams = {}
for i in range(1,9):
    jams[f'region{i}'] = {}
    jam_reg = irreg_data.loc[irreg_data[f'bbox{i}'], 'min_hour':'driverscount']
    jam_hour = {h: jam_reg.loc[(jam_reg['min_hour'] <= h) & (h <= jam_reg['max_hour']), 'length'].sum() for h in range(-2, 24)}
    jams[f'region{i}']['hour_max_jam'] = max(jam_hour.items(), key=operator.itemgetter(1))[0]
    jams[f'region{i}']['max_jam'] = max(jam_hour.items(), key=operator.itemgetter(1))[1]

In [ ]:
for i in range(1,9):
    print(f'total_interactions_reg{i}')

In [316]:
jams

{'region1': {'hour_max_jam': 8, 'max_jam': 44732},
 'region2': {'hour_max_jam': 9, 'max_jam': 20856},
 'region3': {'hour_max_jam': 9, 'max_jam': 6819},
 'region4': {'hour_max_jam': 7, 'max_jam': 613},
 'region5': {'hour_max_jam': 10, 'max_jam': 4106},
 'region6': {'hour_max_jam': 8, 'max_jam': 4229},
 'region7': {'hour_max_jam': 8, 'max_jam': 25341},
 'region8': {'hour_max_jam': -2, 'max_jam': 0}}

In [324]:
jam_reg = irreg_data.loc[:, 'min_hour':'driverscount']
jam_total = {h: jam_reg.loc[(jam_reg['min_hour'] <= h) & (h <= jam_reg['max_hour']), 'length'].sum() for h in range(-2, 24)}

In [328]:
jam_total

{-2: 11190,
 -1: 2903,
 0: 938,
 1: 565,
 2: 1130,
 3: 12443,
 4: 3019,
 5: 13682,
 6: 150774,
 7: 250094,
 8: 750816,
 9: 458408,
 10: 378524,
 11: 202928,
 12: 106356,
 13: 119408,
 14: 68936,
 15: 49357,
 16: 49090,
 17: 110964,
 18: 290684,
 19: 137046,
 20: 23027,
 21: 14077,
 22: 0,
 23: 0}

In [329]:
print(max(jam_total.items(), key=operator.itemgetter(1))[0])
print(max(jam_total.items(), key=operator.itemgetter(1))[1])

8
750816


In [350]:
irreg_data.sort_values(by='length', ascending=False)

id  min_hour  max_hour                                   street  \
127   162400346         6         6                  BR-116 Rod. Pres. Dutra   
21    162403443        11        11               Pista Expressa Marg. Tietê   
7     162421411         8         8             SP-160 N Rod. dos Imigrantes   
365   162400524         7         8                  BR-116 Rod. Pres. Dutra   
24    162404004        12        12               Pista Expressa Marg. Tietê   
5     162420951         8         8              SP-021 Rodoanel Mário Covas   
134   162401964         9         9      SP-070 O Rod. Ayrton Senna da Silva   
48    162402177        11        11      SP-070 O Rod. Ayrton Senna da Silva   
8     162400784         8         8             SP-160 N Rod. dos Imigrantes   
67    162401365         8         8             SP-270 L Rod. Raposo Tavares   
125   162402633        10        10               Pista Expressa Marg. Tietê   
19    162400993         8         8      SP-070 O Rod. Ayrton Senna da Silva   
6     162420350         8         8  BR-116 Rod. Pres. Dutra - Pista Lateral   
62    162401473         8         8               Pista Expressa Marg. Tietê   
214   162422430        10        10                  BR-116 Rod. Pres. Dutra   
176   162400817         8         8                  BR-116 Rod. Pres. Dutra   
57    162401257         8         8               Pista Expressa Marg. Tietê   
28    162400402         6         6               Pista Expressa Marg. Tietê   
583   162420080         3         3             SP-160 N Rod. dos Imigrantes   
3     162406236        18        18               Pista Expressa Marg. Tietê   
429   162401809         9        10                  BR-116 Rod. Pres. Dutra   
43    162402305        10        10            Av. Morvan Dias de Figueiredo   
13    162400779         9         9                    Av. das Nações Unidas   
123   162421860         9         9      SP-070 O Rod. Ayrton Senna da Silva   
58    162400243         6         6                     SP-150 N V. Anchieta   
33    162400824         8         8                BR-381 S Rod. Fernão Dias   
12    162407058        18        18             SP-270 O Rod. Raposo Tavares   
2     162406068        17        17      SP-280 O Rod. Pres. Castello Branco   
44    162400438         8        12  BR-116 Rod. Pres. Dutra - Pista Lateral   
10    162401656         9         9                          Marg. Pinheiros   
...         ...       ...       ...                                      ...   
1382  162405648        15        15             Estr, Pirajussara Valo Velho   
593   162400485         7         7     R. Dr. José Augusto de Souza e Silva   
899   162406735        17        17              R. Joana Pedroso dos Santos   
562   162422000         9         9                         R. São Sebastião   
891   162401929         9         9                         R. São Sebastião   
1278  162422181         9         9                         R. São Sebastião   
1393  162402362         9        11                                      NaN   
1022  162401879         9         9                 R. Da. Germaine Burchard   
770   162401449         8         8             Av. Comendador Dante Carraro   
574   162401343         8         8             Av. Comendador Dante Carraro   
1557  162401129         8         8                          Av. Rui Barbosa   
846   162401162         8         9                 Vd. Fioravante Iervolino   
366   162403637        11        11                              R. do Rocio   
444   162422530        10        10             R. Dr. José Áureo Bustamante   
1172  162402355         9         9             R. Dr. José Áureo Bustamante   
1223  162402396         9         9             R. Dr. José Áureo Bustamante   
527   162422481        10        10             R. Dr. José Áureo Bustamante   
1308  162427050        17        18                           Av. Ellis Maas   
594   1624014

# Distribuição de Acidentes

## Heat Map

In [24]:
cities[city]['clusters'].to_csv('clusters.csv')

# Next Step

In [ ]:
acc = sp['accidents'].loc[sp['accidents']['start_time']> '2018-10-31']

In [ ]:
clusters = wa.clustering(acc, eps_spatial=200, eps_temporal=3600)

In [ ]:
exp = pd.DataFrame(clusters).T.drop(columns=['uuid list'])

In [ ]:
#clusters

# Exploração/Bagunça

## Observando os dados puros

A conclusão da exploração abaixo é que basta pegar dois pontos, inicial e final, e tomar alguns cuidados com as demais colunas. A query pode ser vista no script waze-accidents.py

In [ ]:
city = cities['São Paulo']

city['accidents'] = pd.read_sql_query(f"""select uuid, pub_utc_date, location_x, location_y, subtype,
                                            reliability, confidence, thumbs_up, jam_uuid, report_by_municipality_user
                                            from waze.alerts
                                            where city = '{city['city']}' AND type = 'ACCIDENT'""", con)


In [ ]:
city['accidents']['pub_utc_date'] = city['accidents']['pub_utc_date'].apply(lambda x: x.replace(tzinfo=pytz.utc)
                                                                            .astimezone(city['timezone'])
                                                                            .replace(tzinfo=None))

#city['accidents'].set_index('pub_utc_date', inplace=True)
city['accidents'].rename(columns={'sum': 'length_raw'}, inplace=True)
city['accidents'].sort_index(inplace=True)

In [ ]:
accidents = cities['São Paulo']['accidents']

In [ ]:
accidents['uuid'].agg(['nunique','count','size'])

In [ ]:
accidents.head()

In [ ]:
explore_uuid = accidents.groupby('uuid').nunique()

In [ ]:
accidents.loc[accidents['uuid'] == 'cf56e91d-4b3f-3aec-9a5c-a4edbc10cf82', ['location_x', 'location_y']].groupby(['location_x']).count()

In [ ]:
accidents.loc[accidents['uuid'] == 'cf56e91d-4b3f-3aec-9a5c-a4edbc10cf82', ['location_x', 'location_y']].groupby(['location_y']).count()

In [ ]:
explore_uuid.apply(max,axis=0)

In [ ]:
explore_uuid.loc[explore_uuid['location_x']==5]

In [ ]:
cities = wa.download_accidents_data('São Paulo', cities)